In [ ]:
import boto3
import os

s3 = boto3.client("s3")
bucket = "c144972a3751929l10036967t1w533267341463-bucket-pp6e047xcla7" #enter your bucket name here
input_prefix = "comprehend/input/"
local_dir = "/tmp/comprehend_texts"
os.makedirs(local_dir, exist_ok=True)

uploaded = []

# Upload normalized texts
for _, row in df_transcripts.iterrows():
    if not row["normalized_text"] or row["normalized_text"].startswith("ERROR"):
        continue
    path = f"{local_dir}/{row['job_id']}.txt"
    with open(path, "w") as f:
        f.write(row["normalized_text"])
    key = f"{input_prefix}{row['job_id']}.txt"
    s3.upload_file(path, bucket, key)
    uploaded.append(key)

print(f"Uploaded {len(uploaded)} files.")

# Start Comprehend job
comprehend = boto3.client("comprehend", region_name="us-east-1")
job_name = f"keyphrase-job-{str(uuid.uuid4())[:8]}"
s3_input_uri = f"s3://{bucket}/{input_prefix}"
s3_output_uri = f"s3://{bucket}/comprehend/output/"
role_arn = "your-comprehend-role-arn"

response = comprehend.start_key_phrases_detection_job(
    JobName=job_name,
    DataAccessRoleArn=role_arn,
    InputDataConfig={"S3Uri": s3_input_uri, "InputFormat": "ONE_DOC_PER_FILE"},
    OutputDataConfig={"S3Uri": s3_output_uri},
    LanguageCode="en"
)

print(f"Started Comprehend job: {job_name}")